# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## Note: this is for use with 3d slices of data in Zimmer lab format 

For now I'm testing non-GUI created annotations. Thus, I've deleted the label-creation portions of this demo and replaced them with importing

## Create a new project

It is always good idea to keep the projects seperate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos (for lableing more data) to the project at any stage of the project. 

In [1]:
%env DLClight=True
import deeplabcut

env: DLClight=True
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
task='WormTest' # Enter the name of your experiment Task
experimenter='Charlie' # Enter the name of the experimenter
#video=['wb_immobilized/KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff'] # Enter the paths of your videos OR FOLDER you want to grab frames from.
video=['/groups/zimmer/Kerem Uzel/WB/Cleaned up Datasets/WT/Dataset1_20170626_TKU721_Ctrl_w2/Head/KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff']

#path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=True) 

# NOTE: The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/config.yaml'
#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

In [43]:
# IF using Kerem's data:
#  Also copy the metadata that is generated by MATLAB
from shutil import copyfile
import os

# From the same folder as the video
fname = 'wbstruct.mat'
src = os.path.join(Path(video[0]).parent, 'Quant', fname)

# Just put in the main directory
dst = os.path.join(Path(path_config_file).parent, fname)

#print(src)
#print(dst)
copyfile(src, dst)

'/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/wbstruct.mat'

In [3]:
print(path_config_file)

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/config.yaml


## Now, go edit the config.yaml file that was created! 
Add your body part labels, edit the number of frames to extract per video, etc. 

#### Note that you can see more information about ANY function by adding a ? at the end,  i.e. 

## Save individual volumes from ome.tiff movie

In [6]:
video_fname = '{}\\videos\\KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff'.format(home)

with tifffile.TiffFile(video_fname) as vid:
    #print(vid)
    #tmp = vid.asarray(series=0,key=[1,13])
    #print(vid.ome_metadata)
#    print(vid.ome_metadata.keys())
#    print(vid.ome_metadata['Image']['Pixels'].keys())
    print(vid.ome_metadata['Image']['Pixels']['SizeT'])
    print(vid.ome_metadata['Image']['Pixels']['SizeY'])
    print(vid.ome_metadata['Image']['Pixels']['SizeZ'])
    
#print(tmp.shape)

NameError: name 'home' is not defined

In [38]:
#
# Basically this is a custom extract_frames() function
#

from pathlib import Path
from deeplabcut.utils import auxiliaryfunctions
import tifffile
import platform
import os

def movie2volume(path_config_file, which_vol=None):
    """
    Takes a video filename, which is a large ome-tiff file, and saves a volume in the folder given by 'out_folder'
    """
    
    if platform.system() is 'Windows':
        is_windows = True
    else:
        is_windows = False
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now (will be giant, ~3GB)
    print(video_fname)
    
    # Read basic metadata
    with tifffile.TiffFile(video_fname) as vid:
        #print(tifffile.xml2dict(vid.ome_metadata))
        #print(vid.ome_metadata)
        ome_metadata = vid.ome_metadata
        if isinstance(ome_metadata, str):
            # Appears to be a bug that just returns a string... can fix manually though
            ome_metadata = tifffile.xml2dict(ome_metadata)['OME']
        print(ome_metadata.keys())
        mdat = ome_metadata['Image']['Pixels']
        nz, nt = mdat['SizeZ'], mdat['SizeT']
    
    # Get volume indices to save
    if which_vol is None:
        which_vol = [0]
    
    for i_vol in which_vol:
        print("Read volume {}/{}".format(i_vol, len(which_vol)))
        
        # Convert scalar volume label to the sequential frames
        # Note: this may change for future input videos!
        vol_indices = list(range(i_vol*nz, i_vol*nz + nz))

        # Read and make output name
        this_volume = tifffile.imread(video_fname, key=vol_indices)
        output_name = 'img{}.tif'.format(i_vol)

        # Save in output folder
        fname = Path(video_fname)
        output_path = os.path.join(Path(path_config_file).parents[0],'labeled-data',fname.stem)

        tifffile.imsave(os.path.join(str(output_path),output_name), this_volume)

        print('Saved volume to {}\\{}'.format(output_path, output_name))
    

In [39]:
movie2volume(path_config_file, list(range(1,5)))

Config file read successfully.
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/videos/KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff
dict_keys(['Experimenter', 'Image', '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation'])
Read volume 1/4
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/labeled-data/KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img1.tif
Read volume 2/4
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/labeled-data/KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img2.tif
Read volume 3/4
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/labeled-data/KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img3.tif
Read volume 4/4
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/labeled-data/KU20170626-TKU721_nostim_ctrl_w2_part1.ome\im

# Import annotations from MATLAB

In [44]:
import h5py
import os
import pandas as pd
import glob
import numpy as np
from pathlib import Path
from deeplabcut.utils import auxiliaryfunctions


In [45]:
# For now, just have the labels in the example folder
home = os.path.dirname(path_config_file)
label_fname = '{}\\wbstruct.mat'.format(home)
with h5py.File(label_fname, 'r') as mat:
    #print(mat.keys())
    #print(mat['mask'].shape)
    #print(mat['simple'].keys())
    print(mat['simple']['x'].shape)
    print(mat['simple']['x'][0,:])
    print(mat['simple']['x'][0,:].shape)
    #print(mat['simple']['y'][:])
    #print(mat['simple']['z'].shape)
    print(max(mat['simple']['z']))
    print(mat['simple']['nn'][0][0])


OSError: Unable to open file (unable to open file: name = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12\wbstruct.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [53]:
def wb_tracker2dlc_format(path_config_file):
    """
    Converts Zimmer Whole-Brain tracker data for immobilized worms into training data for DeepLabCut
    """
    
    # Build filenames
    home = os.path.dirname(path_config_file)
    wb_fname = os.path.join(home,'wbstruct.mat')
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    # Note: the labeled-data subfolder has the entire video name as the folder name
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now
    fname = Path(video_fname)
    output_path = os.path.join(Path(path_config_file).parents[0],'labeled-data',fname.stem)
    output_path = output_path
    print('Looking in folder {}'.format(output_path))
    
    # Get list of images
    imlist=[]
    imtype = '*.tif'
    imlist.extend([fn for fn in glob.glob(os.path.join(output_path,imtype)) if ('labeled.png' not in fn)])

    if len(imlist)==0:
        print("No images found!!")

    index = np.sort(imlist)
    print('Working on folder: {}'.format(os.path.split(str(output_path))[-1]))
    relativeimagenames=['labeled'+n.split('labeled')[1] for n in index]
    
    # Build dataset using pandas; copied from from labeling_toolbox.py
    scorer = cfg['scorer']
    with h5py.File(wb_fname, 'r') as mat:
        num_neurons = int(mat['simple']['nn'][0][0])
        x = mat['simple']['x'][:,0] # NOTE: a flip is required here; not sure why
        y = mat['simple']['y'][:,0]
        z = mat['simple']['z'][:]
        coords = np.empty((len(index),3,))
        dataFrame = None
        
        for i in range(num_neurons):
            bodypart = 'neuron{}'.format(i)
             
            # Note: this requires a flip in the y direction; not sure why
            x_sz = 272
            coords[:] = np.array([x_sz-x[i], y[i], z[i][0]])

            index = pd.MultiIndex.from_product([[scorer], [bodypart], ['x', 'y', 'z']],names=['scorer', 'bodyparts', 'coords'])
            #print(index)
            #print(coords)
            frame = pd.DataFrame(coords, columns = index, index = relativeimagenames)
            dataFrame = pd.concat([dataFrame, frame],axis=1)
        
    dataFrame.to_csv(os.path.join(output_path,"CollectedData_" + scorer + ".csv"))
    dataFrame.to_hdf(os.path.join(output_path,"CollectedData_" + scorer + '.h5'),'df_with_missing',format='table', mode='w')
    
    print('Finished')

In [54]:
wb_tracker2dlc_format(path_config_file);

Config file read successfully.
Looking in folder /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/labeled-data/KU20170626-TKU721_nostim_ctrl_w2_part1.ome
Working on folder: KU20170626-TKU721_nostim_ctrl_w2_part1.ome
Finished


# Automatically update the config file

In [55]:
import csv

def wb_tracker2config_names(path_config_file):
    """
    Automatically updates the config file with the proper number of neurons, and deletes any other default bodyparts.
    Only affects the "bodyparts" field
    """
    
    # Get number of neurons from wbstruct
    home = os.path.dirname(path_config_file)
    wb_fname = os.path.join(home,'wbstruct.mat')
    with h5py.File(wb_fname, 'r') as mat:
        num_neurons = int(mat['simple']['nn'][0][0])
    
    # Read in entire config file into a list
    config_rows = []
    with open(path_config_file) as config:
        c_reader = csv.reader(config)#, delimiter=' ')
        for row in c_reader:
            config_rows.append(row)
    
    ## Delete the current bodypart lines
    delete_these_rows = False
    config_rows_edit = config_rows.copy()
    for row in config_rows:
        if row == ['bodyparts:']:
            delete_these_rows = True # Start deleting next row
        elif row == ['start: 0']:
            delete_these_rows = False # Do not delete this row, or others
            break
        elif delete_these_rows == True:
            # Don't delete either of the two above, but only in between those rows
            config_rows_edit.remove(row)
    
    ## Add in the named neuron lines
    # Using "list slicing" https://www.geeksforgeeks.org/python-insert-list-in-another-list/
    new_names = [['- neuron{}'.format(i)] for i in range(num_neurons)]
    insert_index = config_rows_edit.index(['start: 0'])
    config_rows_edit[insert_index:insert_index] = new_names
    
    ## Write the file again
    if True:
        with open(path_config_file, 'w', newline='') as config:
            c_writer = csv.writer(config)
            for row in config_rows_edit:
                c_writer.writerow(row)
    #for row in config_rows_edit:
        #c_writer.writerow(row)
     #   print(row[:])
    
    

In [56]:
wb_tracker2config_names(path_config_file)

## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [3]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Charlie.
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/labeled-data/KU20170626-TKU721_nostim_ctrl_w2_part1.ome_labeled  already exists!


100%|██████████| 4/4 [00:00<00:00, 57.14it/s]


If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


UnboundLocalError: local variable 'image' referenced before assignment

If the labels need adjusted, you can use relauch the labeling GUI to move them around, save, and re-plot!

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [1]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/config.yaml'
#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

env: DLClight=True


/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/users/

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
deeplabcut.create_training_dataset(path_config_file)
#remember, there are several networks you can pick, the default is resnet-50!

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/training-datasets/iteration-2/UnaugmentedDataSet_WormTestJun12  already exists!
Recognized z-slice training data; using custom function
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95, 1, (array([0, 2, 1]), array([3])))]

In [202]:
deeplabcut.create_training_dataset?

## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [1]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/config.yaml'

env: DLClight=True


/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/users/

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
deeplabcut.train_network(path_config_file, displayiters=50, saveiters=1000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33],
                [34],
                [35],
                [36],
                [37],
                [38],
                [39],
                [40],
                [41],
                [42],
                [43],
                [44],
            

Selecting single-animal trainer
Switching batchsize to 1, as default/tensorpack/deterministic loaders do not support batches >1. Use imgaug loader.
Starting with standard pose-dataset loader.
Initializing PoseNetSlices
Creating new class for use with z-slice data, PoseNetSlices
Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 50
Save_iters overwritten as 1000
Training parameter:
{'using_z_slices': True, 'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/dlc-models/iteration-2/WormTestJun12-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 1, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 're

iteration: 50 loss: 0.6110 lr: 0.005
iteration: 100 loss: 0.0324 lr: 0.005
iteration: 150 loss: 0.0260 lr: 0.005
iteration: 200 loss: 0.0218 lr: 0.005
iteration: 250 loss: 0.0184 lr: 0.005
iteration: 300 loss: 0.0157 lr: 0.005
iteration: 350 loss: 0.0130 lr: 0.005
iteration: 400 loss: 0.0112 lr: 0.005
iteration: 450 loss: 0.0102 lr: 0.005
iteration: 500 loss: 0.0090 lr: 0.005
iteration: 550 loss: 0.0082 lr: 0.005
iteration: 600 loss: 0.0076 lr: 0.005
iteration: 650 loss: 0.0073 lr: 0.005
iteration: 700 loss: 0.0068 lr: 0.005
iteration: 750 loss: 0.0066 lr: 0.005
iteration: 800 loss: 0.0065 lr: 0.005
iteration: 850 loss: 0.0063 lr: 0.005
iteration: 900 loss: 0.0060 lr: 0.005
iteration: 950 loss: 0.0060 lr: 0.005
iteration: 1000 loss: 0.0061 lr: 0.005
iteration: 1050 loss: 0.0056 lr: 0.005
iteration: 1100 loss: 0.0055 lr: 0.005
iteration: 1150 loss: 0.0054 lr: 0.005
iteration: 1200 loss: 0.0053 lr: 0.005
iteration: 1250 loss: 0.0053 lr: 0.005
iteration: 1300 loss: 0.0051 lr: 0.005
iterat

InvalidArgumentError: LossTensor is inf or nan : Tensor had NaN values
	 [[node train_op/CheckNumerics (defined at /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/deeplabcut/pose_estimation_tensorflow/train.py:127) ]]
	 [[node train_op/control_dependency (defined at /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/deeplabcut/pose_estimation_tensorflow/train.py:127) ]]

Caused by op 'train_op/CheckNumerics', defined at:
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instan

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [3]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/config.yaml'

#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

env: DLClight=True


In [4]:
# from IPython.core.debugger import set_trace
# set_trace()
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33],
                [34],
                [35],
                [36],
                [37],
                [38],
                [39],
                [40],
                [41],
                [42],
                [43],
                [44],
            

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/evaluation-results/  already exists!
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/evaluation-results/iteration-2/WormTestJun12-trainset95shuffle1  already exists!
Running  DLC_resnet50_WormTestJun12shuffle1_10000  with # of trainingiterations: 10000
Setting up Volume-based evaluation
Initializing PoseNetSlices
Creating new class for use with z-slice data, PoseNetSlices


0it [00:00, ?it/s]

Analyzing data...
> /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/deeplabcut/pose_estimation_tensorflow/nnet/predict.py(91)argmax_pose_predict()
     89     from IPython.core.debugger import set_trace
     90     set_trace()
---> 91     for joint_idx in range(num_joints):
     92         maxloc = np.unravel_index(
     93             np.argmax(scmap[:, :, joint_idx]), scmap[:, :, joint_idx].shape



ipdb>  c


...

ipdb>  c


4it [00:08,  2.00s/it]
  0%|          | 0/4 [00:00<?, ?it/s]

Done and results stored for snapshot:  snapshot-10000
Results for 10000  training iterations: 95 1 train error: 44.98 pixels. Test error: 43.06  pixels.
With pcutoff of 0.6  train error: 46.61 pixels. Test error: 47.96 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|██████████| 4/4 [00:04<00:00,  1.04s/it]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [3]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

Using snapshot-11000 for model /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/dlc-models/iteration-0/WormTestJun12-trainset95shuffle1
Initializing ResNet
No video(s) were found. Please check your paths and/or 'video_type'.


'DLC_resnet50_WormTestJun12shuffle1_11000'

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [3]:
deeplabcut.extract_outlier_frames?

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

# Aside: access an internal object

In [17]:
# Try to understand the crash below by accessing the internal dataset object
from deeplabcut.pose_estimation_tensorflow.dataset.factory import (
    create as create_dataset,
)

from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.utils.auxiliaryfunctions import read_config
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch
from easydict import EasyDict as edict
import os
from pathlib import Path

In [24]:
cfg = edict(read_config(path_config_file))

path_pose_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/dlc-models/iteration-1/WormTestJun12-trainset95shuffle1/train/pose_cfg.yaml'
cfg_pose = load_config(path_pose_config_file)

In [25]:
dataset = create_dataset(cfg_pose)

Starting with standard pose-dataset loader.


FileNotFoundError: [Errno 2] No such file or directory: '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/dlc-models/iteration-1/WormTestJun12-trainset95shuffle1/train/training-datasets/iteration-1/UnaugmentedDataSet_WormTestJun12/WormTest_Charlie95shuffle1.mat'

In [ ]:
batch_spec = get_batch_spec(cfg)
batch, enqueue_op, placeholders = setup_preloading(batch_spec)